# PROJECT PROPOSAL GROUP 17

## Title/ Inferential Question

### Introduction

-insert intro here-


### Data Analysis

In [15]:
#Load this before continuing
library(tidyverse)
library(digest)
library(infer)
library(repr)
library(tidyverse)

In [11]:
#Reading data into R
#_df <- read_csv("abc.csv")
#head(_df)

### Methods

-insert methods here-

### References

-insert ref here-